In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os
from datetime import datetime, timedelta
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [12]:
# Create an output directory if it doesn't exist
output_dir = 'rice_planting_dss_output'
os.makedirs(output_dir, exist_ok=True)

In [13]:
def save_plt(filename):
    """Save the current plot to the output directory."""
    plt.savefig(f'{output_dir}/{filename}.png', bbox_inches='tight', dpi=300)
    plt.close()

In [14]:
def load_data(file_path):
    """Load and prepare BMKG data with proper date handling."""
    try:
        # Read data with Date column present
        data = pd.read_csv(file_path)
        print(f"Raw data shape: {data.shape}")
        
        # Check if necessary columns exist
        required_columns = ['Date', 'TN', 'TX', 'TAVG', 'RH_AVG', 'RR']
        missing_columns = [col for col in required_columns if col not in data.columns]
        
        if missing_columns:
            print(f"Warning: Missing required columns: {missing_columns}")
            
        # Check if Date column exists, if not try to create it from Year, Month, Day
        if 'Date' not in data.columns and all(col in data.columns for col in ['Year', 'Month', 'Day']):
            print("Creating Date column from Year, Month, Day columns")
            data['Date'] = pd.to_datetime(data[['Year', 'Month', 'Day']])
        
        # Convert Date to datetime and set as index
        if 'Date' in data.columns:
            data['Date'] = pd.to_datetime(data['Date'])
            data.set_index('Date', inplace=True)
            print(f"Successfully loaded data with shape: {data.shape}")
            
            # Display sample of loaded data
            print("Sample of loaded data:")
            print(data.head(3))
            return data
        else:
            print("Error: No Date column found or could be created")
            return None
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        return None

In [15]:
def preprocess_data(data):
    """Clean and preprocess the data with improved methods."""
    # List columns that need to be fixed
    cols_to_fix = ['TN', 'TX', 'TAVG', 'RH_AVG', 'RR', 'SS', 'FF_X', 'DDD_X', 'FF_AVG']
    
    # Clean: replace '-' with NaN, then convert to numeric
    for col in cols_to_fix:
        if col in data.columns:
            data[col] = pd.to_numeric(data[col], errors='coerce')
    
    # Handle 8888 and 9999 values (missing value codes)
    for col in data.columns:
        if data[col].dtype != 'object':  # Only change numeric columns
            data[col] = data[col].replace([8888, 9999], np.nan)

    # Check missing values percentage
    missing_percentage = data.isna().mean() * 100
    print("Missing Values Percentage by Column:")
    for col, pct in missing_percentage.items():
        print(f"{col}: {pct:.2f}%")
    
    # Create a copy for filled data
    data_filled = data.copy()
    
    # Improved missing value handling
    # For RR (rainfall), use time-based interpolation
    if 'RR' in data_filled.columns:
        # First interpolate using time-based method
        data_filled['RR'] = data_filled['RR'].interpolate(method='time')
        # Then fill any remaining NaNs
        data_filled['RR'] = data_filled['RR'].fillna(method='ffill').fillna(method='bfill')
        # Ensure no values are <= 0 for multiplicative models
        data_filled['RR'] = data_filled['RR'].clip(lower=0.1)
    
    # For other columns, use standard ffill/bfill
    for col in data_filled.columns:
        if col != 'RR':  # Skip RR as we already handled it
            data_filled[col] = data_filled[col].fillna(method='ffill').fillna(method='bfill')
    
    return data, data_filled

In [16]:
# Function to analyze seasonal patterns
def analyze_seasonality(data, variables):
    """Analyze seasonal patterns in time series data using ACF/PACF."""
    for var in variables:
        if var in data.columns:
            plt.figure(figsize=(12, 6))
            plot_acf(data[var].dropna(), lags=400, alpha=0.05)
            plt.title(f'Autocorrelation Function for {var}')
            save_plt(f'acf_{var}')
            
            plt.figure(figsize=(12, 6))
            plot_pacf(data[var].dropna(), lags=50, alpha=0.05)
            plt.title(f'Partial Autocorrelation Function for {var}')
            save_plt(f'pacf_{var}')
            
            print(f"Saved ACF/PACF plots for {var}")

In [ ]:
# Function to perform Holt-Winters forecasting with improved parameters
def forecast_variable(data, variable, period, forecast_days=120, seasonal_type='mul'):
    """
    Apply Holt-Winters forecasting to a specific variable with improved parameters.
    
    Parameters:
    -----------
    data : pandas.DataFrame
        The input data with the variable to forecast
    variable : str
        The column name to forecast
    period : int
        Seasonal period (7=weekly, 30=monthly, 365=yearly)
    forecast_days : int
        Number of days to forecast
    seasonal_type : str
        Type of seasonal component ('mul' or 'add')
    
    Returns:
    --------
    tuple
        (model, forecast values, fitted values)
    """
    try:
        print(f"\nForecasting {variable} with seasonal period: {period} ({seasonal_type} seasonality)")
        
        # Check if we have enough data points for the given period
        if len(data) < 2 * period:
            print(f"Warning: Data length ({len(data)}) is less than twice the seasonal period ({period})")
        
        # Create and fit the model
        model = ExponentialSmoothing(
            data[variable],
            trend='add',  # Always use additive trend
            seasonal=seasonal_type,  # Use parameter for flexibility
            seasonal_periods=period
        )
        
        # For RH_AVG, we can set some parameters directly if needed
        if variable == 'RH_AVG':
            fitted_model = model.fit(optimized=True, 
                                    smoothing_level=0.5,  # Make it more responsive to level changes
                                    smoothing_seasonal=0.1)  # Increase seasonal component impact
        else:
            fitted_model = model.fit(optimized=True)
        
        # Generate forecast
        forecast = fitted_model.forecast(forecast_days)
        
        # Get fitted values
        fitted = fitted_model.fittedvalues
        
        # Calculate metrics
        mse = mean_squared_error(data[variable].iloc[period:], fitted.iloc[period:])
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(data[variable].iloc[period:], fitted.iloc[period:])
        r2 = r2_score(data[variable].iloc[period:], fitted.iloc[period:])
        
        print(f"Model parameters:")
        print(f"  Alpha (level): {fitted_model.params['smoothing_level']:.4f}")
        print(f"  Beta (trend): {fitted_model.params['smoothing_trend']:.4f}")
        print(f"  Gamma (seasonal): {fitted_model.params['smoothing_seasonal']:.4f}")
        print(f"Model performance:")
        print(f"  MSE: {mse:.4f}")
        print(f"  RMSE: {rmse:.4f}")
        print(f"  MAE: {mae:.4f}")
        print(f"  R-squared: {r2:.4f}")
        
        return fitted_model, forecast, fitted
        
    except Exception as e:
        print(f"Error forecasting {variable}: {str(e)}")
        return None, None, None


In [ ]:
# Classification functions for the three main variables
def classify_rr(value):
    """Classify rainfall values."""
    if pd.isna(value):  # Handle NaN values
        return 'Tidak Ada Data (Risiko)'
    elif value < 2:
        return 'Kering (Risiko)'
    elif 2 <= value <= 15:
        return 'Optimal'
    else:
        return 'Banjir (Risiko)'

def classify_tavg(value):
    """Classify average temperature values."""
    if pd.isna(value):  # Handle NaN values
        return 'Tidak Ada Data (Risiko)'
    elif value < 20:
        return 'Dingin (Risiko)'
    elif 20 <= value <= 35:
        return 'Optimal'
    else:
        return 'Panas (Risiko)'

def classify_rh(value):
    """Classify relative humidity values."""
    if pd.isna(value):  # Handle NaN values
        return 'Tidak Ada Data (Risiko)'
    elif value < 60:
        return 'Kering (Risiko)'
    elif 60 <= value <= 90:
        return 'Optimal'
    else:
        return 'Lembab Ekstrem (Risiko)'

In [ ]:
# Calculate decision score and recommendation
def calculate_decision(rr_value, tavg_value, rh_value, ss_value=None):
    """
    Calculate decision score and recommendation based on weather parameters.
    
    Parameters:
    -----------
    rr_value : float
        Rainfall value in mm
    tavg_value : float
        Average temperature in °C
    rh_value : float
        Relative humidity in %
    ss_value : float, optional
        Sunshine duration in hours
    
    Returns:
    --------
    tuple
        (score, category, decision)
    """
    # Check if we have valid data
    if pd.isna(rr_value) and pd.isna(tavg_value) and pd.isna(rh_value):
        return 0, 'Tidak Ada Data', 'Bera'
    
    # Get classifications
    rr_status = classify_rr(rr_value)
    tavg_status = classify_tavg(tavg_value)
    rh_status = classify_rh(rh_value)
    
    # Initialize score
    score = 0
    
    # Apply weighted scoring: RR (40%), TAVG (40%), RH_AVG (20%)
    # Rainfall scoring
    if not pd.isna(rr_value):  # Only score if we have data
        if 'Optimal' in rr_status:
            score += 40
        elif 'Kering' in rr_status and rr_value >= 1:  # Some rain is better than none
            score += 20
    
    # Temperature scoring
    if not pd.isna(tavg_value):  # Only score if we have data
        if 'Optimal' in tavg_status:
            score += 40
        elif tavg_value >= 18 and tavg_value < 20:  # Close to optimal
            score += 30
        elif tavg_value > 35 and tavg_value <= 37:  # Close to optimal
            score += 30
    
    # Humidity scoring
    if not pd.isna(rh_value):  # Only score if we have data
        if 'Optimal' in rh_status:
            score += 20
        elif rh_value > 90 and rh_value <= 95:  # Slightly high but manageable
            score += 10
    
    # Build category description
    category_parts = []
    if 'Optimal' not in rr_status:
        category_parts.append(rr_status)
    if 'Optimal' not in tavg_status:
        category_parts.append(tavg_status)
    if 'Optimal' not in rh_status:
        category_parts.append(rh_status)
    
    if category_parts:
        category = ', '.join(category_parts)
    else:
        category = 'Optimal'
    
    # Determine decision
    if ('Banjir' in rr_status and not pd.isna(rr_value)) or \
       ('Panas' in tavg_status and not pd.isna(tavg_value)) or \
       (('Lembab Ekstrem' in rh_status) and not pd.isna(rh_value) and not pd.isna(rr_value) and rr_value > 10):
        decision = 'Bera'  # Don't plant due to high risk
    elif score >= 70:
        decision = 'Tanam'  # Optimal conditions for planting
    elif 50 <= score < 70:
        decision = 'Tanam (Waspada)'  # Plant with caution
    else:
        decision = 'Bera'  # Don't plant due to suboptimal conditions
    
    return score, category, decision

In [ ]:
def check_harvest_conditions(forecast_df, planting_date):
    """
    Check conditions for harvesting based on planting date.
    
    Parameters:
    -----------
    forecast_df : pandas.DataFrame
        DataFrame with forecasted values
    planting_date : str
        Planting date in YYYY-MM-DD format
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame with harvesting recommendations
    """
    # Convert planting date to datetime
    plant_date = pd.to_datetime(planting_date)
    
    # Define harvest window (90-100 days after planting)
    harvest_start = plant_date + pd.Timedelta(days=90)
    harvest_end = plant_date + pd.Timedelta(days=100)
    
    # Get forecast for harvest window plus a few days before and after
    buffer_days = 7
    window_start = harvest_start - pd.Timedelta(days=buffer_days)
    window_end = harvest_end + pd.Timedelta(days=buffer_days)
    
    # Extract relevant forecast period
    harvest_forecast = forecast_df[(forecast_df.index >= window_start) & 
                                  (forecast_df.index <= window_end)].copy()
    
    # Check for risky conditions during harvest
    harvest_forecast['Rainfall_Risk'] = harvest_forecast['RR'] > 10
    harvest_forecast['Humidity_Risk'] = harvest_forecast['RH_AVG'] > 90
    
    # Add recommendation column
    def get_harvest_recommendation(row):
        if row['Rainfall_Risk']:
            if row.name >= harvest_start and row.name <= harvest_end:
                return 'Percepat Panen (Risiko Hujan)'
            else:
                return 'Pantau Cuaca'
        elif row['Humidity_Risk']:
            if row.name >= harvest_start and row.name <= harvest_end:
                return 'Pertimbangkan Panen (Kelembaban Tinggi)'
            else:
                return 'Pantau Kelembaban'
        elif row.name >= harvest_start and row.name <= harvest_end:
            return 'Panen Sesuai Jadwal'
        else:
            return 'Belum Waktunya Panen'
    
    harvest_forecast['Rekomendasi'] = harvest_forecast.apply(get_harvest_recommendation, axis=1)
    
    return harvest_forecast


In [ ]:
def visualize_forecasts(original_data, forecast_df, variable, period):
    """Create visualization for forecasting results."""
    plt.figure(figsize=(12, 6))
    
    # Plot historical data (last 365 days)
    historical_data = original_data[variable].iloc[-365:]
    plt.plot(historical_data.index, historical_data, 
             label='Data Historis', color='gray', alpha=0.7)
    
    # Plot forecast
    plt.plot(forecast_df.index, forecast_df[variable], 
             label=f'Forecast (period={period})', 
             color='blue', linewidth=2)
    
    # Add confidence intervals
    plt.fill_between(forecast_df.index, 
                     forecast_df[f'{variable}_Lower'], 
                     forecast_df[f'{variable}_Upper'], 
                     color='blue', alpha=0.2)
    
    plt.title(f'Forecast {variable} dengan Seasonal Period {period} Hari')
    plt.xlabel('Tanggal')
    plt.ylabel(variable)
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    save_plt(f'forecast_{variable}_period_{period}')

In [ ]:
def visualize_decisions(decision_df):
    """Create visualization for the decision support results."""
    # Plot decisions timeline
    plt.figure(figsize=(14, 8))
    
    # Convert decision to numeric for plotting
    decision_map = {'Bera': 0, 'Tanam (Waspada)': 1, 'Tanam': 2}
    decision_df['Decision_Value'] = decision_df['Keputusan'].map(
        lambda x: decision_map.get(x, 0) if 'Panen' not in x else 3
    )
    
    # Plot score
    plt.subplot(2, 1, 1)
    plt.plot(decision_df.index, decision_df['Skor'], 
             marker='o', linestyle='-', markersize=4)
    plt.fill_between(decision_df.index, 0, decision_df['Skor'], alpha=0.3)
    plt.axhline(y=70, color='green', linestyle='--', alpha=0.7, label='Batas Optimal (70%)')
    plt.axhline(y=50, color='orange', linestyle='--', alpha=0.7, label='Batas Waspada (50%)')
    plt.title('Skor Keputusan untuk Penanaman Padi')
    plt.ylabel('Skor (%)')
    plt.ylim(0, 105)
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    # Plot decisions
    plt.subplot(2, 1, 2)
    colors = ['red', 'orange', 'green', 'blue']
    decision_colors = [colors[val] for val in decision_df['Decision_Value']]
    
    plt.scatter(decision_df.index, decision_df['Decision_Value'], 
                c=decision_colors, s=50)
    plt.yticks([0, 1, 2, 3], ['Bera', 'Tanam (Waspada)', 'Tanam', 'Panen'])
    plt.title('Rekomendasi Penanaman Padi')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    save_plt('rice_planting_decisions_timeline')
    
    # Create a second visualization for parameter distributions
    try:
        # Get unique decisions and skip plotting if we don't have at least one value for each
        unique_decisions = decision_df['Keputusan'].unique()
        
        if len(unique_decisions) > 0:
            plt.figure(figsize=(16, 12))
            
            # Use manual plotting instead of seaborn for more control
            plt.subplot(2, 2, 1)
            for i, decision in enumerate(unique_decisions):
                subset = decision_df[decision_df['Keputusan'] == decision]['RR'].dropna()
                if len(subset) > 0:  # Only plot if we have data
                    plt.boxplot(subset, positions=[i+1], widths=0.6)
            plt.xticks(range(1, len(unique_decisions)+1), unique_decisions)
            plt.title('Curah Hujan Berdasarkan Keputusan')
            plt.axhline(y=2, color='orange', linestyle='--', label='Batas Kering (2mm)')
            plt.axhline(y=15, color='red', linestyle='--', label='Batas Banjir (15mm)')
            plt.legend()
            
            # Temperature by decision
            plt.subplot(2, 2, 2)
            for i, decision in enumerate(unique_decisions):
                subset = decision_df[decision_df['Keputusan'] == decision]['TAVG'].dropna()
                if len(subset) > 0:  # Only plot if we have data
                    plt.boxplot(subset, positions=[i+1], widths=0.6)
            plt.xticks(range(1, len(unique_decisions)+1), unique_decisions)
            plt.title('Suhu Rata-rata Berdasarkan Keputusan')
            plt.axhline(y=20, color='blue', linestyle='--', label='Batas Dingin (20°C)')
            plt.axhline(y=35, color='red', linestyle='--', label='Batas Panas (35°C)')
            plt.legend()
            
            # Humidity by decision
            plt.subplot(2, 2, 3)
            for i, decision in enumerate(unique_decisions):
                subset = decision_df[decision_df['Keputusan'] == decision]['RH_AVG'].dropna()
                if len(subset) > 0:  # Only plot if we have data
                    plt.boxplot(subset, positions=[i+1], widths=0.6)
            plt.xticks(range(1, len(unique_decisions)+1), unique_decisions)
            plt.title('Kelembaban Relatif Berdasarkan Keputusan')
            plt.axhline(y=60, color='orange', linestyle='--', label='Batas Kering (60%)')
            plt.axhline(y=90, color='blue', linestyle='--', label='Batas Lembab (90%)')
            plt.legend()
            
            # Score by decision
            plt.subplot(2, 2, 4)
            for i, decision in enumerate(unique_decisions):
                subset = decision_df[decision_df['Keputusan'] == decision]['Skor'].dropna()
                if len(subset) > 0:  # Only plot if we have data
                    plt.boxplot(subset, positions=[i+1], widths=0.6)
            plt.xticks(range(1, len(unique_decisions)+1), unique_decisions)
            plt.title('Distribusi Skor Berdasarkan Keputusan')
            
            plt.tight_layout()
            save_plt('decision_parameters_distribution')
        else:
            print("Warning: No unique decisions found for boxplot visualization")
    except Exception as e:
        print(f"Error in boxplot visualization: {str(e)}")
        # Create a simple figure with error message
        plt.figure(figsize=(10, 6))
        plt.text(0.5, 0.5, f'Visualisasi boxplot gagal: {str(e)}',
                ha='center', va='center', fontsize=12)
        plt.tight_layout()
        save_plt('decision_parameters_distribution_error')
        

In [ ]:
def main():
    """Main function to run the Rice Planting Decision Support System."""
    print("=" * 80)
    print("SISTEM PENDUKUNG KEPUTUSAN PENANAMAN PADI BERBASIS FORECASTING HOLT-WINTERS")
    print("=" * 80)
    
    # 1. Load and preprocess data
    print("\nSTEP 1: Loading and preprocessing data...")
    data_path = input("Enter the path to the BMKG data CSV file: ")
    data = load_data(data_path)
    
    if data is None:
        print("Failed to load data. Exiting.")
        return
    
    raw_data, filled_data = preprocess_data(data)
    
    # 1a. Analyze seasonal patterns before forecasting
    print("\nSTEP 1a: Analyzing seasonal patterns...")
    analyze_seasonality(filled_data, ['RR', 'TAVG', 'RH_AVG'])
    
    # 2. Perform Holt-Winters forecasting for each selected variable
    print("\nSTEP 2: Forecasting weather variables...")
    
    # Variables and their seasonal periods (updated based on analysis)
    forecast_variables = {
        'RR': (365, 'add'),   # Rainfall (yearly seasonality, additive to handle zeros)
        'TAVG': (7, 'add'),   # Average temperature (weekly seasonality, changed from 30)
        'RH_AVG': (3, 'add')  # Relative humidity (changed from 7 to 3 days)
    }
    
    # Prepare forecasting
    forecast_days = 120  # 4 months forecast
    forecast_start_date = filled_data.index[-1] + pd.Timedelta(days=1)
    forecast_dates = pd.date_range(start=forecast_start_date, periods=forecast_days, freq='D')
    
    # Initialize forecast dataframe
    forecast_results = pd.DataFrame(index=forecast_dates)
    
    # Run forecasts for each variable
    models = {}
    for var, (period, seasonal_type) in forecast_variables.items():
        if var in filled_data.columns:
            model, forecast, fitted = forecast_variable(filled_data, var, period, forecast_days, seasonal_type)
            
            if model is not None:
                # Store in the forecast results
                forecast_results[var] = forecast
                
                # Calculate confidence intervals (95%)
                mse = mean_squared_error(filled_data[var].iloc[period:], fitted.iloc[period:])
                rmse = np.sqrt(mse)
                
                forecast_results[f'{var}_Lower'] = forecast - 1.96 * rmse
                forecast_results[f'{var}_Upper'] = forecast + 1.96 * rmse
                
                # Ensure no negative values in forecast
                if var == 'RR':
                    forecast_results[var] = forecast_results[var].clip(lower=0)
                    forecast_results[f'{var}_Lower'] = forecast_results[f'{var}_Lower'].clip(lower=0)
                
                # Store model for later use
                models[var] = model
                
                # Visualize forecast
                visualize_forecasts(filled_data, forecast_results, var, period)
        else:
            print(f"Warning: Variable {var} not found in the dataset.")
    
    # 3. Check if sunshine duration (SS) is available
    if 'SS' in filled_data.columns:
        # Use a simpler method for SS forecasting (e.g., monthly seasonality)
        model_ss, forecast_ss, fitted_ss = forecast_variable(filled_data, 'SS', 30, forecast_days, 'add')
        if model_ss is not None:
            forecast_results['SS'] = forecast_ss
            
            # Calculate confidence intervals
            mse_ss = mean_squared_error(filled_data['SS'].iloc[30:], fitted_ss.iloc[30:])
            rmse_ss = np.sqrt(mse_ss)
            
            forecast_results['SS_Lower'] = forecast_ss - 1.96 * rmse_ss
            forecast_results['SS_Upper'] = forecast_ss + 1.96 * rmse_ss
            
            # Visualize forecast
            visualize_forecasts(filled_data, forecast_results, 'SS', 30)
    
    # Print a preview of forecast results
    print("\nPreview of forecast results:")
    print(forecast_results.head())  # Debug line to check forecast values
    
    # 4. Apply decision support logic
    print("\nSTEP 3: Applying decision support logic...")
    
    # Create a copy of the forecast results for decision making
    decision_df = forecast_results.copy()
    
    # Set negative values to 0 (can't have negative rainfall, etc.)
    for var in forecast_variables.keys():
        if var in decision_df.columns:
            decision_df[var] = decision_df[var].clip(lower=0)
    
    # Apply the decision logic for each day in the forecast
    decision_df['Skor'] = 0
    decision_df['Kategori'] = ''
    decision_df['Keputusan'] = ''
    
    # Debug: count valid forecast days
    valid_days = 0
    
    for idx, row in decision_df.iterrows():
        try:
            # Check if we have the required variables (even if NaN)
            required_vars = list(forecast_variables.keys())
            missing_vars = [var for var in required_vars if var not in row.index]
            
            if missing_vars:
                print(f"Warning: Missing variables {missing_vars} for date {idx}")
                continue
                
            # Get values, may be NaN
            rr_val = row['RR'] if 'RR' in row else np.nan
            tavg_val = row['TAVG'] if 'TAVG' in row else np.nan
            rh_val = row['RH_AVG'] if 'RH_AVG' in row else np.nan
            ss_val = row.get('SS', np.nan)
            
            # Debug print to check values
            if idx.day == 1 or valid_days < 5:  # Print first few days or 1st of each month
                print(f"Day {idx}: RR={rr_val:.2f}, TAVG={tavg_val:.2f}, RH={rh_val:.2f}")
                valid_days += 1
            
            # Calculate decision using updated function that handles NaN
            score, category, decision = calculate_decision(rr_val, tavg_val, rh_val, ss_val)
            
            # Store results
            decision_df.at[idx, 'Skor'] = score
            decision_df.at[idx, 'Kategori'] = category
            decision_df.at[idx, 'Keputusan'] = decision
        except Exception as e:
            print(f"Error calculating decision for {idx}: {str(e)}")
    
    # 5. Check harvesting conditions
    print("\nSTEP 4: Checking harvesting conditions...")
    planting_date = input("Enter a reference planting date (YYYY-MM-DD) or press Enter to skip: ")
    
    if planting_date:
        try:
            harvest_forecast = check_harvest_conditions(decision_df, planting_date)
            
            # Apply harvesting decisions to main decision dataframe
            for idx, row in harvest_forecast.iterrows():
                if 'Panen' in row['Rekomendasi']:
                    decision_df.at[idx, 'Keputusan'] = row['Rekomendasi']
        except Exception as e:
            print(f"Error checking harvest conditions: {str(e)}")
    
    # 6. Visualize and save results
    print("\nSTEP 5: Visualizing and saving results...")
    visualize_decisions(decision_df)
    
    # 7. Save results to CSV with proper column naming
    output_file = f'{output_dir}/rice_planting_decisions.csv'
    decision_columns = ['RR', 'TAVG', 'RH_AVG', 'Skor', 'Kategori', 'Keputusan']
    if 'SS' in decision_df.columns:
        decision_columns.insert(3, 'SS')
    
    # Create output dataframe with Tanggal column first
    decision_output = decision_df[decision_columns].copy()
    decision_output.reset_index(inplace=True)
    decision_output.rename(columns={'index': 'Tanggal'}, inplace=True)
    
    # Reorganize columns to ensure Tanggal is first
    final_columns = ['Tanggal'] + decision_columns
    decision_output = decision_output[final_columns]
    
    # Save to CSV without index
    decision_output.to_csv(output_file, index=False)
    
    print(f"\nDecision support results saved to {output_file}")
    
    # Display sample of results with proper column ordering
    print("\nSample of Decision Support Results:")
    print(decision_output.head(10))
    
    print("\n" + "=" * 80)
    print("RICE PLANTING DECISION SUPPORT SYSTEM COMPLETED")
    print("=" * 80)

if __name__ == "__main__":
    main()